In [2]:
import tensorflow as tf
from tensorflow import keras

#   継承を使用したモデル構築(図10-16のモデルを構築する)
#   call()によるブラックボックス化がデメリット
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
    
model = WideAndDeepModel()
        
        

In [ ]:
#   モデルの保存と復元
#[...]   #   モデルの構築、コンパイル、訓練
model.save("my_keras_model.h5")  #   HDF5形式によるモデルの保存
model = keras.model.load_model("my_keras.model.h5")

In [ ]:
#   コールバック関数を使用したエポックごとの保存
#[...]  #   モデルの構築、コンパイル
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras.model.h5")
history = model.fit(X_train, y_train, epochs=10, callbacks=[checkpoint_cb]) #   引数にsave_best_only=Trueと指定することで
                                                                            #   検証セットに対する評価が最高になったときだけ保存

In [ ]:
#   コールバック関数を使用した早期打ち切り
#   patienceだけ検証セットに対する性能が上がらないときに終了
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                    restore_best_weights=True)  #   終了時のではなく最良の重みを保存
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb])


In [ ]:
#   カスタムコールバックの実装(デバッグ用などに)
#   例：訓練中の評価ロスと訓練ロスの比率を計算して過学習を検知
class PrintValTrainRatioCallback(kera.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:,2f}".format(logs["val_loss"]/logs["loss"]))
#   ほかにも様々なタイミングで使える関数を定義できる

In [ ]:
#   tensorboardを使用した可視化
import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_log_dir = get_run_logdir()  #   形式：./my_logs/run_2019_06_07-15_15_22

[...]   #   モデルの構築、コンパイル
tensorboard_cb = keras.callbacks.TensorBoard(run_log_dir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data =(X_valid, y_valid),
                    callbacks=[tensorboard_cb])